# A3: Project on complex networks
### Analyzing Spotify Data: Global Patterns of Musical Taste and Artist/Genre Clustering by Country
#### _By Saioa Elizondo, Marc Albesa and Maria Fité_
**Group Name:** A3L

**Objective:** Development of a Project in which you put into practice some of the knowledge acquired
during this course on Complex Networks.

**Kind of project:** Analysis. Obtain real data which can be put in terms of network(s),
and analyze the network(s) to answer any question or hypothesis
about them. It is not enough to calculate descriptors or communities,
you must try to answer relevant questions about the system's
functioning.

## 0. Preparing the enviroment

In [5]:
%pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [17]:
# Code to import the needed modules/packages to run the notebook
import os
import ast
import pandas as pd # dataframes
import numpy as np # Mathematical operations
import matplotlib.pyplot as plt # Representation
import random as random
import networkx as nx # Network tools
import spotipy
from matplotlib import rcParams
from matplotlib import colors
from matplotlib.lines import Line2D
from matplotlib.patches import FancyArrowPatch
from collections import Counter
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigsh
from spotipy.oauth2 import SpotifyClientCredentials

# Get workng directory
wd = os.getcwd()
np.random.seed(52)

In [18]:
# Set up credentials
client_id = 'albesa.marc@gmail.com'
client_secret = 'Pauimarc.06'  

# Authenticate
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

## 1. Importing data and constructing the network

In [20]:
playlist_ids = {
    "Catalonia": "5IczcOjRojDGQmphylNgAk",  # Regional playlist for Catalonia
    "Spain": "37i9dQZEVXbNFJfN1Vw8d9",  # Top 50 - Spain
    "Argentina": "37i9dQZEVXbMMy2roB9myp",  # Top 50 - Argentina
    "Brazil": "37i9dQZEVXbMXbN3EUUhlg",  # Top 50 - Brazil
    "Italy": "37i9dQZEVXbIQnj7RRhdSX",  # Top 50 - Italy
    "France": "37i9dQZEVXbIPWwFssbupI",  # Top 50 - France
    "USA": "37i9dQZEVXbLRQDuF5jeBp",  # Top 50 - USA
    "Colombia": "37i9dQZEVXbOa2lmxNORXQ",  # Top 50 - Colombia
    "UK": "37i9dQZEVXbLnolsZ8PSNw",  # Top 50 - UK
    "Puerto Rico": "37i9dQZEVXbJvfa0Yxg7E7",  # Top 50 - Puerto Rico
    "Mexico": "37i9dQZEVXbO3qyFxbkOE1",  # Top 50 - Mexico
    "Chile": "37i9dQZEVXbL0GavIqMTeb",  # Top 50 - Chile
    "Russia": "37i9dQZEVXbL8l7ra5vVdB",  # Top 50 - Russia
    "Germany": "37i9dQZEVXbJiZcmkrIHGU",  # Top 50 - Germany
    "Portugal": "37i9dQZEVXbKyJS56d1pgi",  # Top 50 - Portugal
    "South Korea": "37i9dQZEVXbNxXF4SkHj9F",  # Top 50 - South Korea
    "Venezuela": "37i9dQZEVXbJ6IpvItkve3",  # Top 50 - Venezuela
    "Morocco": "37i9dQZEVXbJxGz0rDIpGj",  # Top 50 - Morocco
    "China": "37i9dQZEVXbMVY2FDHm6NN",  # Top 50 - China
    "Romania": "37i9dQZEVXbNZbJ6TZelCq",  # Top 50 - Romania
    "Sweden": "37i9dQZEVXbLoATJ81JYXz",  # Top 50 - Sweden
    "India": "37i9dQZEVXbLZ52XmnySJg",  # Top 50 - India
    "Uruguay": "37i9dQZEVXbMJJi3wgRbAy",  # Top 50 - Uruguay
    "Japan": "37i9dQZEVXbKXQ4mDTEBXq",  # Top 50 - Japan
    "Ireland": "37i9dQZEVXbKM896FDX8L1",  # Top 50 - Ireland
    "Cuba": "37i9dQZEVXbJp9wcIM9Eo5",  # Top 50 - Cuba
    "Australia": "37i9dQZEVXbJPcfkRz0wJ0",  # Top 50 - Australia
}

In [21]:
dataframe = []
dataframe.append(list(playlist_ids.keys()))

countries_and_playlists = [("Catalonia", "5IczcOjRojDGQmphylNgAk")]

# Fetch playlist tracks
for country, playlist_id in countries_and_playlists:
    playlist_tracks = sp.playlist_items(playlist_id)
    country_artists = {"Country": country}

# Print each track's position and details
    for index, item in enumerate(playlist_tracks['items']):
        track = item["track"]
        Position = {index + 1}
        Artist = {track["artists"][0]["name"]}
        Album = track["album"]
        Popularity = track["popularity"]
        popularity_score = Popularity/Position
        country_artists["Artist"] = popularity_score

# Convert to DataFrame for analysis
df = pd.DataFrame(dataframe)

SpotifyOauthError: error: invalid_client, error_description: Invalid client

In [11]:
dataframe = []

df = pd.DataFrame(columns=["Country"])

print(df)

# Initial DataFrame
df = pd.DataFrame([
    {"Country": "USA", "Artist": "Artist1", "Score": 95},
    {"Country": "Canada", "Artist": "Artist2", "Score": 90}
])

# New data for an existing artist
new_data = {"Country": "USA", "Artist": "Artist1", "Score": 98}

# Check if the artist already exists
if new_data["Artist"] in df["Artist"].values:
    df.loc[df["Artist"] == new_data["Artist"], ["Country", "Score"]] = new_data["Country"], new_data["Score"]
else:
    # Add as a new row
    df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)

print(df)


# Initialize the DataFrame with artists as columns
df = pd.DataFrame(columns=["Country", "Artist1", "Artist2", "Artist3"])

# New data for a country
new_data = {"Country": "USA", "Artist1": 95, "Artist2": 90, "Artist3": 80}

# Check if the country already exists in the DataFrame
if new_data["Country"] in df["Country"].values:
    # Update the existing row
    df.loc[df["Country"] == new_data["Country"], :] = new_data
else:
    # Add a new row
    df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)

print(df)

[<generator object <genexpr> at 0x15b003580>]

In [ ]:
# Reading csv files
edges = pd.read_csv(os.path.join(wd, "network", "edges.csv"))
nodes = pd.read_csv(os.path.join(wd, "network", "nodes.csv"))

# Defining directed graph
G = nx.DiGraph()

# Adding nodes
nodenames = nodes["# index"].values
    
for node in nodenames:
    G.add_node(node, label=node)

# Adding edges
for i in range(len(edges)):
    node1 = edges["# source"][i]
    node2 = edges[" target"][i]
    regulation = int(edges[" regulation_type"][i])
    if regulation == 1:
        G.add_edge(node1,node2, reg_type = "1")
    elif regulation == 2:
        G.add_edge(node2,node1, reg_type = "2")
    elif regulation == 3:
        G.add_edge(node1,node2, reg_type = "3")
        G.add_edge(node2,node1, reg_type = "3")

# If the network is not (weakly) connected, select the largest connected component (LCC) 
# for the rest of the analysis. This LCC must have at least 200 nodes.

# Get all weakly connected components
weak_components = list(nx.weakly_connected_components(G)) 

# Find the largest weakly connected component (LCC)
largest_weak_component = max(weak_components, key=len)
G_lcc = G.subgraph(largest_weak_component).copy()

# Updating nodes and edges for the lcc
nodes_lcc = nodes[ nodes["# index"].isin(G_lcc.nodes())]
edges_lcc = edges[edges['# source'].isin(G_lcc.nodes()) & edges[' target'].isin(G_lcc.nodes())]

# Check connectivity of the largest weakly connected component
print("Largest connected component is weakly connected:",nx.is_weakly_connected(G_lcc))

# Check number of nodes is at least 200
print('The number of nodes is:', G_lcc.number_of_nodes())

In [ ]:
df.to_csv('spotify_top50_data.csv', index=False)

In [ ]:
df.to_csv('spotify_top50_data.csv', index=False)